<a href="https://colab.research.google.com/github/iramvai007/ApachePySparkInBigData/blob/main/CSE488__All_About_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSE488 
## Spark Tutorial in pySpark

In this tutorial you will learn how to use [Apache Spark](https://spark.apache.org) in local mode on a Colab enviroment.

Credits to [Tiziano Piccardi](http://piccardi.me/) for his Spark Tutorial used in the Applied Data Analysis class at EPFL.

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!apt-get update

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install graphframes


openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 85 not upgraded.


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell"

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1L6pCQkldvdBoaEhRFzL0VnrggEFvqON4'
downloaded = drive.CreateFile({'id': id}) 
downloaded.GetContentFile('Bombing_Operations.json.gz')

id='14dyBmcTBA32uXPxDbqr0bFDIzGxMTWwl'
downloaded = drive.CreateFile({'id': id}) 
downloaded.GetContentFile('Aircraft_Glossary.json.gz')  

If you executed the cells above, you should be able to see the files *Bombing_Operations.json.gz* and *Aircraft_Glossary.json.gz* under the "Files" tab on the left panel.

In [ ]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.


In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

NameError: ignored

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [ ]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-12-26 11:12:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  8.45MB/s    in 1.6s    

2021-12-26 11:12:41 (8.45 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


# Vietnam War

**Pres. Johnson**: _What do you think about this Vietnam thing? I’d like to hear you talk a little bit._

**Sen. Russell**: _Well, frankly, Mr. President, it’s the damn worse mess that I ever saw, and I don’t like to brag and I never have been right many times in my life, but I knew that we were going to get into this sort of mess when we went in there._

May 27, 1964

![banner](https://raw.githubusercontent.com/epfl-ada/2019/c17af0d3c73f11cb083717b7408fedd86245dc4d/Tutorials/04%20-%20Scaling%20Up/img/banner.jpg)

----

The Vietnam War, also known as the Second Indochina War, and in Vietnam as the Resistance War Against America or simply the American War, was a conflict that occurred in Vietnam, Laos, and Cambodia from 1 November 1955 to the fall of Saigon on 30 April 1975. It was the second of the Indochina Wars and was officially fought between North Vietnam and the government of South Vietnam.

**The dataset describes all the air force operation in during the Vietnam War.**

**Bombing_Operations** [Get the dataset here](https://drive.google.com/a/epfl.ch/file/d/1L6pCQkldvdBoaEhRFzL0VnrggEFvqON4/view?usp=sharing)

- AirCraft: _Aircraft model (example: EC-47)_
- ContryFlyingMission: _Country_
- MissionDate: _Date of the mission_
- OperationSupported: _Supported War operation_ (example: [Operation Rolling Thunder](https://en.wikipedia.org/wiki/Operation_Rolling_Thunder))
- PeriodOfDay: _Day or night_
- TakeoffLocation: _Take off airport_
- TimeOnTarget
- WeaponType
- WeaponsLoadedWeight

**Aircraft_Glossary** [Get the dataset here](https://drive.google.com/a/epfl.ch/file/d/14dyBmcTBA32uXPxDbqr0bFDIzGxMTWwl/view?usp=sharing)

- AirCraft: _Aircraft model (example: EC-47)_
- AirCraftName
- AirCraftType

**Dataset Information:**

THOR is a painstakingly cultivated database of historic aerial bombings from World War I through Vietnam. THOR has already proven useful in finding unexploded ordnance in Southeast Asia and improving Air Force combat tactics:
https://www.kaggle.com/usaf/vietnam-war-bombing-operations

Load the datasets:

In [ ]:
Bombing_Operations = spark.read.json("Bombing_Operations.json.gz")
Aircraft_Glossary = spark.read.json("Aircraft_Glossary.json.gz")

In [ ]:
Bombing_Operations.printSchema()

root
 |-- AirCraft: string (nullable = true)
 |-- ContryFlyingMission: string (nullable = true)
 |-- MissionDate: string (nullable = true)
 |-- OperationSupported: string (nullable = true)
 |-- PeriodOfDay: string (nullable = true)
 |-- TakeoffLocation: string (nullable = true)
 |-- TargetCountry: string (nullable = true)
 |-- TimeOnTarget: double (nullable = true)
 |-- WeaponType: string (nullable = true)
 |-- WeaponsLoadedWeight: long (nullable = true)



In [ ]:
Aircraft_Glossary.printSchema()

root
 |-- AirCraft: string (nullable = true)
 |-- AirCraftName: string (nullable = true)
 |-- AirCraftType: string (nullable = true)



In [ ]:
Bombing_Operations.show()

+--------+--------------------+-----------+------------------+-----------+---------------+-------------+------------+--------------------+-------------------+
|AirCraft| ContryFlyingMission|MissionDate|OperationSupported|PeriodOfDay|TakeoffLocation|TargetCountry|TimeOnTarget|          WeaponType|WeaponsLoadedWeight|
+--------+--------------------+-----------+------------------+-----------+---------------+-------------+------------+--------------------+-------------------+
|   EC-47|UNITED STATES OF ...| 1971-06-05|              null|          D|   TAN SON NHUT|     CAMBODIA|      1005.0|                null|                  0|
|   EC-47|UNITED STATES OF ...| 1972-12-26|              null|          D|  NAKHON PHANOM|SOUTH VIETNAM|       530.0|                null|                  0|
|    RF-4|UNITED STATES OF ...| 1973-07-28|              null|          D|       UDORN AB|         LAOS|       730.0|                null|                  0|
|     A-1|UNITED STATES OF ...| 1970-02-02|   

In [ ]:
Aircraft_Glossary.show()

+--------+--------------------+--------------------+
|AirCraft|        AirCraftName|        AirCraftType|
+--------+--------------------+--------------------+
|     A-1|Douglas A-1 Skyra...|         Fighter Jet|
|    A-26|Douglas A-26 Invader|        Light Bomber|
|    A-37|Cessna A-37 Drago...|Light ground-atta...|
|     A-4|McDonnell Douglas...|         Fighter Jet|
|     A-5|North American A-...|          Bomber Jet|
|     A-6|Grumman A-6 Intruder|     Attack Aircraft|
|     A-7|  LTV A-7 Corsair II|     Attack Aircraft|
|  AC-119|Fairchild AC-119 ...|Military Transpor...|
|  AC-123|Fairchild C-123 P...|Military Transpor...|
|  AC-130|Lockheed AC-130 S...|Fixed wing ground...|
|   AC-47|Douglas AC-47 Spooky|Ground attack air...|
|    AH-1| Bell AH-1 HueyCobra|          Helicopter|
|     B-1| Rockwell B-1 Lancer|Heavy strategic b...|
|    B-52| B-52 Stratofortress|    Strategic bomber|
|    B-57|Martin B-57 Canberra|     Tactical Bomber|
|    B-66|Douglas B-66 Dest...|        Light B

In [ ]:
Aircraft_Glossary.take(5)

[Row(AirCraft='A-1', AirCraftName='Douglas A-1 Skyraider', AirCraftType='Fighter Jet'),
 Row(AirCraft='A-26', AirCraftName='Douglas A-26 Invader', AirCraftType='Light Bomber'),
 Row(AirCraft='A-37', AirCraftName='Cessna A-37 Dragonfly', AirCraftType='Light ground-attack aircraft'),
 Row(AirCraft='A-4', AirCraftName='McDonnell Douglas A-4 Skyhawk', AirCraftType='Fighter Jet'),
 Row(AirCraft='A-5', AirCraftName='North American A-5 Vigilante', AirCraftType='Bomber Jet')]

##projecting colums


In [ ]:
Aircraft_Glossary.select("AirCraft", "AirCraftName").show()


+--------+--------------------+
|AirCraft|        AirCraftName|
+--------+--------------------+
|     A-1|Douglas A-1 Skyra...|
|    A-26|Douglas A-26 Invader|
|    A-37|Cessna A-37 Drago...|
|     A-4|McDonnell Douglas...|
|     A-5|North American A-...|
|     A-6|Grumman A-6 Intruder|
|     A-7|  LTV A-7 Corsair II|
|  AC-119|Fairchild AC-119 ...|
|  AC-123|Fairchild C-123 P...|
|  AC-130|Lockheed AC-130 S...|
|   AC-47|Douglas AC-47 Spooky|
|    AH-1| Bell AH-1 HueyCobra|
|     B-1| Rockwell B-1 Lancer|
|    B-52| B-52 Stratofortress|
|    B-57|Martin B-57 Canberra|
|    B-66|Douglas B-66 Dest...|
|     C-1| Grumman C-1A Trader|
|   C-117|     C-117D Skytrain|
|   C-119|Fairchild C-119 F...|
|   C-123|Fairchild C-123 P...|
+--------+--------------------+
only showing top 20 rows



##selecting tuples

In [ ]:
Aircraft_Glossary.where(Aircraft_Glossary.AirCraft=='A-1').show() #it shows the result in another format or it's a formatted output 

+--------+--------------------+------------+
|AirCraft|        AirCraftName|AirCraftType|
+--------+--------------------+------------+
|     A-1|Douglas A-1 Skyra...| Fighter Jet|
+--------+--------------------+------------+



In [ ]:
Aircraft_Glossary.where("AirCraft=='A-1'").collect() #it shows in raw format

[Row(AirCraft='A-1', AirCraftName='Douglas A-1 Skyraider', AirCraftType='Fighter Jet')]

##counting the number of rows

In [ ]:
Bombing_Operations.count()

4400775

In [ ]:
x=Aircraft_Glossary.count()

In [ ]:
print("there are %d",x, "rows")

there are %d 104 rows


##Writing the Queries

**1.Find the aircraft model and the mission date for the operations place over LAOS (target country)**

**2.Find the Aircraft model, the mission date and take off location for the operation took place over LAOS and the duration was more than 1000**

In [ ]:
##sol 1
df_q1= Bombing_Operations.where("TargetCountry = 'LAOS'").select("AirCraft", "MissionDate")

In [ ]:
df_q1.show()


NameError: ignored

In [ ]:
df_q1.count()

In [ ]:
#sol 2
df_q2= Bombing_Operations.where("TargetCountry = 'LAOS' AND TimeOnTarget> 1000 ").select("AirCraft", "MissionDate", "TakeoffLocation")


In [ ]:
df_q2.show()

In [ ]:
df_q2.count()

##Aggeregates

**Number of missios for each counrty operationg the missions (CountryFlyingMissions)**

In [ ]:
Bombing_Operations.groupBy("ContryFlyingMission").agg(count("*")).show()

In [ ]:
df_q3=Bombing_Operations.groupBy("ContryFlyingMission").agg(count("*").alias("Number of missions"))

In [ ]:
df_q3.show()

In [ ]:
df_q3.sort("Number of missions").show()

In [ ]:
df_q3.sort(desc("Number of missions")).show()

**Find the number of Operations for each Aircraft Type**

In [ ]:
join_results= Bombing_Operations.join(Aircraft_Glossary, Bombing_Operations.AirCraft == Aircraft_Glossary.AirCraft)

In [ ]:
Bombing_Operations.show()

In [ ]:
missionsPerAircraftType=join_results.groupBy("AirCraftType").agg(count("*").alias("NumOperations")).sort("AirCraftType")
%time missionsPerAircraftType.show()

**Caching the result**

In [ ]:
missionsPerAircraftType.cache()

In [ ]:
missionsPerAircraftType.show()

Saving the result into a file

In [ ]:
%time missionsPerAircraftType.write.mode('overwrite').json("missionsPerAirCraftType.json")

In [ ]:
print(missionsPerAircraftType.count())

##Creating RDDs from Differenrt Types of Data

In [ ]:
data = [("Alice", 29, 14500), ("BOB", 22, 12000),("Charlie", 35, 45000)]
colums = ["Name", "Age", "Salary"]

In [ ]:
rdd1 = sc.parallelize(data)

In [ ]:
print(rdd1.collect())

##RDD to DataFrame Conversion

In [ ]:
dfFromRDD1 = rdd1.toDF(colums)

In [ ]:
dfFromRDD1.printSchema()
dfFromRDD1.show()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Salary: long (nullable = true)

+-------+---+------+
|   Name|Age|Salary|
+-------+---+------+
|  Alice| 29| 14500|
|    BOB| 22| 12000|
|Charlie| 35| 45000|
+-------+---+------+



In [ ]:
lines = sc.textFile("people.txt")
print(type(lines))
print(lines.collect())

<class 'pyspark.rdd.RDD'>
['Alice,45,4500', 'Bob,59,130000', 'Dooffy,45,50000']


In [ ]:
rows= lines.map(lambda value: value.split(","))
print(type(rows))
print(rows.collect())

<class 'pyspark.rdd.PipelinedRDD'>
[['Alice', '45', '4500'], ['Bob', '59', '130000'], ['Dooffy', '45', '50000']]


In [ ]:
peopleRDD = rows.map(lambda value: Row(name = value[0], age = value[1], salary = float(value[2])))
print(type(peopleRDD))
print(peopleRDD.collect())

<class 'pyspark.rdd.PipelinedRDD'>
[Row(name='Alice', age='45', salary=4500.0), Row(name='Bob', age='59', salary=130000.0), Row(name='Dooffy', age='45', salary=50000.0)]


In [ ]:
dfFromPeople = peopleRDD.toDF()
dfFromPeople.show()

+------+---+--------+
|  name|age|  salary|
+------+---+--------+
| Alice| 45|  4500.0|
|   Bob| 59|130000.0|
|Dooffy| 45| 50000.0|
+------+---+--------+



##Dataframe from CSV file

In [ ]:
dfFromCSV = spark.read.options(header = "True", inferSchema = "True" ).csv("people.csv")

In [ ]:
dfFromCSV.show()

+-------+----+
|   Name| Age|
+-------+----+
|Michael|29.0|
|   Andy|30.0|
| Justin|19.0|
+-------+----+



In [ ]:
list1 = [i for i in range(11)]
print(list1)

In [ ]:
list1RDD = sc.parallelize(list1)
print(list1RDD.collect())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
intermediate1 = list1RDD.map(lambda value: (value))
print(intermediate1.collect())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
# from operator import add
final = intermediate1.reduce(lambda x,y: (x+y))
print(final)

55


In [ ]:
from operator import add
final1 = intermediate1.reduce(add)
print(final1)

##it has to be a single indivisual value not a tupel (x,y)

55


##Word Count Problem using Spark

In [ ]:
list2 = ['Apple', 'Apple', 'Cat', 'Dog', 'Bat','Apple']
rdd_list2 = sc.parallelize(list2)

In [ ]:
rdd_map_list2 = rdd_list2.map(lambda w: (w, 1))
print(rdd_map_list2.collect())

[('Apple', 1), ('Apple', 1), ('Cat', 1), ('Dog', 1), ('Bat', 1), ('Apple', 1)]


In [ ]:
rdd_reduce_list2 = rdd_map_list2.reduceByKey(lambda x,y : x+y)
print(rdd_reduce_list2.collect()) 

[('Bat', 1), ('Apple', 3), ('Cat', 1), ('Dog', 1)]


In [ ]:
rdd_reduce_list2 = rdd_map_list2.reduceByKey(lambda x,y : x+y).sortBy(lambda r: r[0])
print(rdd_reduce_list2.collect()) 

[('Apple', 3), ('Bat', 1), ('Cat', 1), ('Dog', 1)]


##WordCount from the txt file

In [ ]:
words = sc.textFile("WordCount.txt")
print(type(words))
print(words.collect())

<class 'pyspark.rdd.RDD'>
['From fairest creatures we desire increase,', "That thereby beauty's rose might never die,", 'But as the riper should by time decease,', 'His tender heir might bear his memory:', 'But thou contracted to thine own bright eyes,', "Feed'st thy light's flame with self-substantial fuel,", 'Making a famine where abundance lies,', 'Thy self thy foe, to thy sweet self too cruel:', "Thou that art now the world's fresh ornament,", 'And only herald to the gaudy spring,', 'Within thine own bud buriest thy content,', "And tender churl mak'st waste in niggarding:", 'Pity the world, or else this glutton be,', "To eat the world's due, by the grave and thee."]


In [ ]:
row= words.map(lambda value: value.split(" "))
print(type(row))
print(row.collect())

<class 'pyspark.rdd.PipelinedRDD'>
[['From', 'fairest', 'creatures', 'we', 'desire', 'increase,'], ['That', 'thereby', "beauty's", 'rose', 'might', 'never', 'die,'], ['But', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease,'], ['His', 'tender', 'heir', 'might', 'bear', 'his', 'memory:'], ['But', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes,'], ["Feed'st", 'thy', "light's", 'flame', 'with', 'self-substantial', 'fuel,'], ['Making', 'a', 'famine', 'where', 'abundance', 'lies,'], ['Thy', 'self', 'thy', 'foe,', 'to', 'thy', 'sweet', 'self', 'too', 'cruel:'], ['Thou', 'that', 'art', 'now', 'the', "world's", 'fresh', 'ornament,'], ['And', 'only', 'herald', 'to', 'the', 'gaudy', 'spring,'], ['Within', 'thine', 'own', 'bud', 'buriest', 'thy', 'content,'], ['And', 'tender', 'churl', "mak'st", 'waste', 'in', 'niggarding:'], ['Pity', 'the', 'world,', 'or', 'else', 'this', 'glutton', 'be,'], ['To', 'eat', 'the', "world's", 'due,', 'by', 'the', 'grave', 'and', 'thee.']]


In [ ]:
rdd_word = sc.parallelize(row)
rdd_map_word = rdd_word.map(lambda w: (w, 1))
print(rdd_map_word.collect())

TypeError: ignored

**Using Flat Map**

## Spark and ML

In [ ]:
import pandas as pd


In [ ]:
data = pd.read_excel("transactions.xlsx")

In [ ]:
data.head()

,SalesDate,SalesValue,SalesAmount,Customer,SalesTransactionID,SalesItem
0,2018-09-28,8280.0,10,0,0,0
1,2018-09-28,7452.0,10,0,0,0
2,2019-04-23,21114.0,30,0,1,0
3,2019-04-23,7038.0,10,0,1,1
4,2019-04-23,7000.0,2,0,1,2


In [ ]:
sparkdata = spark.createDataFrame(data)

In [ ]:
sparkdata.show()

+-------------------+----------+-----------+--------+------------------+---------+
|          SalesDate|SalesValue|SalesAmount|Customer|SalesTransactionID|SalesItem|
+-------------------+----------+-----------+--------+------------------+---------+
|2018-09-28 00:00:00|    8280.0|         10|       0|                 0|        0|
|2018-09-28 00:00:00|    7452.0|         10|       0|                 0|        0|
|2019-04-23 00:00:00|   21114.0|         30|       0|                 1|        0|
|2019-04-23 00:00:00|    7038.0|         10|       0|                 1|        1|
|2019-04-23 00:00:00|    7000.0|          2|       0|                 1|        2|
|2019-02-08 00:00:00|     745.2|          1|       0|                 2|        1|
|2019-02-08 00:00:00|    6334.2|          9|       0|                 2|        1|
|2019-02-22 00:00:00|   14076.0|         20|       0|                 3|        0|
|2018-12-11 00:00:00|    7452.0|         10|       0|                 4|        0|
|201

##droping the duplicates

In [ ]:
basketdata = sparkdata.dropDuplicates(['SalesTransactionID', 'SalesItem']).sort('SalesTransactionID')

In [ ]:
basketdata.show()

+-------------------+----------+-----------+--------+------------------+---------+
|          SalesDate|SalesValue|SalesAmount|Customer|SalesTransactionID|SalesItem|
+-------------------+----------+-----------+--------+------------------+---------+
|2018-09-28 00:00:00|    8280.0|         10|       0|                 0|        0|
|2019-04-23 00:00:00|   21114.0|         30|       0|                 1|        0|
|2019-04-23 00:00:00|    7038.0|         10|       0|                 1|        1|
|2019-04-23 00:00:00|    7000.0|          2|       0|                 1|        2|
|2019-02-08 00:00:00|     745.2|          1|       0|                 2|        1|
|2019-02-22 00:00:00|   14076.0|         20|       0|                 3|        0|
|2018-12-11 00:00:00|    7452.0|         10|       0|                 4|        0|
|2018-11-21 00:00:00|    6706.8|          9|       0|                 5|        0|
|2018-09-21 00:00:00|    7000.0|          2|       0|                 6|        2|
|201

In [ ]:
from pyspark.sql import functions as F 

In [ ]:
basketdata = basketdata.groupBy("SalesTransactionID").agg(F.collect_list("SalesItem").alias("items")).sort("SalesTransactionID")

In [ ]:
basketdata.show()

+------------------+--------------------+
|SalesTransactionID|               items|
+------------------+--------------------+
|                 0|                 [0]|
|                 1|           [0, 1, 2]|
|                 2|                 [1]|
|                 3|                 [0]|
|                 4|                 [0]|
|                 5|                 [0]|
|                 6|                 [2]|
|                 7|                 [2]|
|                 8|                 [0]|
|                10|              [1, 0]|
|                11|                 [0]|
|                12|              [4, 3]|
|                13|                 [5]|
|                15|           [7, 8, 9]|
|                16|[17, 15, 16, 20, ...|
|                17|                [21]|
|                18|                [22]|
|                19|[33, 41, 27, 34, ...|
|                20|[59, 50, 57, 60, ...|
|                21|                [62]|
+------------------+--------------

##Finding Frequent Itemsets

**FP Growth Approch**

In [ ]:
from pyspark.ml.fpm import FPGrowth

In [ ]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.01, minConfidence=0.01)
model = fpGrowth.fit(basketdata)

In [ ]:
model.freqItemsets.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+----------+----+
|     items|freq|
+----------+----+
|     [315]| 553|
|     [137]| 730|
|     [363]| 512|
|     [159]| 671|
|     [131]| 599|
|     [135]| 690|
|     [302]| 622|
|     [152]| 761|
|     [146]| 524|
|     [128]| 929|
|     [226]| 842|
|     [312]|1063|
|[312, 108]| 610|
|[312, 103]| 572|
|      [72]| 567|
|     [102]| 874|
|     [222]| 585|
|      [68]| 968|
|     [108]|1655|
|[108, 110]| 510|
+----------+----+
only showing top 20 rows



**Association Rules**

In [ ]:
model.associationRules.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+----------+----------+-------------------+------------------+--------------------+
|antecedent|consequent|         confidence|              lift|             support|
+----------+----------+-------------------+------------------+--------------------+
|      [64]|      [63]| 0.6592082616179001|24.110890692680083| 0.01581794903564201|
|      [64]|      [20]| 0.4208261617900172| 7.183266729236296|0.010097881303432041|
|      [64]|     [316]|0.43459552495697074|21.109050041691336|0.010428282327675216|
| [67, 103]|     [104]| 0.6595208070617906|13.213882748355099|0.010799983479948788|
|[108, 104]|     [103]|  0.740029542097489|16.880202828833255|0.010345682071614423|
| [92, 103]|      [91]| 0.7162162162162162|22.318845872899928|0.010944533928055177|
| [92, 104]|      [91]| 0.7405159332321699|23.076077595045728|0.010077231239416843|
| [104, 20]|      [67]| 0.5626911314984709|13.796901637440483|0.011398835336389542|
| [104, 20]|     [103]|  0.583078491335372|13.300122007257055|0.011811836616

Lift = P(A Intersection B)/p(A) * P(B)
Lift > 1 means A and B togerther appear more than expected 

In [ ]:
rules = model.associationRules
frequentItemsets = model.freqItemsets

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
rules.filter(rules.lift <10).show()

+----------+----------+-------------------+-----------------+--------------------+
|antecedent|consequent|         confidence|             lift|             support|
+----------+----------+-------------------+-----------------+--------------------+
|      [64]|      [20]| 0.4208261617900172|7.183266729236296|0.010097881303432041|
|     [229]|     [104]| 0.3439306358381503|6.890850215597132|0.012286788089043077|
|     [229]|     [103]| 0.3069364161849711|7.001273146572497|0.010965183992070375|
|     [229]|      [20]| 0.2976878612716763|5.081364952394147|0.010634782967827201|
|     [109]|     [104]| 0.4348958333333333|8.713390825403392|0.010345682071614423|
|     [108]|     [110]| 0.3081570996978852|6.870541302932683|0.010531532647751208|
|     [108]|      [67]|0.35891238670694864|8.800349994263643|0.012266138025027878|
|     [108]|     [104]|0.40906344410876133| 8.19582389094368|0.013980093338289349|
|     [108]|      [20]| 0.3595166163141994|6.136747148971244|0.012286788089043077|
|   

In [ ]:
frequentItemsets.filter( "freq>2000").show()

+-----+----+
|items|freq|
+-----+----+
|[110]|2172|
| [20]|2837|
|[104]|2417|
|[103]|2123|
+-----+----+



Prediction

In [ ]:
transformed = model.transform(basketdata)
transformed.show()

+------------------+--------------------+--------------------+
|SalesTransactionID|               items|          prediction|
+------------------+--------------------+--------------------+
|                 0|                 [0]|                  []|
|                 1|           [0, 1, 2]|                  []|
|                 2|                 [1]|                  []|
|                 3|                 [0]|                  []|
|                 4|                 [0]|                  []|
|                 5|                 [0]|                  []|
|                 6|                 [2]|                  []|
|                 7|                 [2]|                  []|
|                 8|                 [0]|                  []|
|                10|              [1, 0]|                  []|
|                11|                 [0]|                  []|
|                12|              [4, 3]|                  []|
|                13|                 [5]|              

In [ ]:
a = 104
transformed.filter(array_contains(transformed.items, 104)).show()



+------------------+--------------------+--------------------+
|SalesTransactionID|               items|          prediction|
+------------------+--------------------+--------------------+
|                25|[7, 86, 105, 104,...|[312, 108, 122, 1...|
|                27|[20, 6, 141, 114,...|[103, 67, 312, 91...|
|                35|[118, 85, 82, 17,...|[67, 312, 110, 98...|
|                36|[92, 108, 153, 91...|[103, 312, 110, 6...|
|                39|[103, 158, 209, 2...|[312, 110, 98, 12...|
|                46|[176, 185, 250, 8...|[108, 83, 122, 12...|
|                52|[93, 270, 168, 13...|[91, 203, 312, 11...|
|                58|[203, 208, 122, 2...|[312, 110, 91, 98...|
|                61|[20, 122, 280, 84...|[312, 110, 91, 98...|
|                66|[160, 392, 13, 67...|[103, 108, 110, 9...|
|                68|[306, 216, 20, 10...|[103, 312, 110, 9...|
|                70|[303, 84, 94, 9, ...|[108, 203, 109, 1...|
|                72|[109, 291, 201, 1...|[64, 312, 110,

Finding the predictions for [104]

In [ ]:
transformed.where(transformed.items == array(*(lit(x) for x in [104]))).show()

+------------------+-----+--------------------+
|SalesTransactionID|items|          prediction|
+------------------+-----+--------------------+
|              3752|[104]|[108, 109, 110, 8...|
|              3867|[104]|[108, 109, 110, 8...|
|              3882|[104]|[108, 109, 110, 8...|
|              3920|[104]|[108, 109, 110, 8...|
|              3925|[104]|[108, 109, 110, 8...|
|              3934|[104]|[108, 109, 110, 8...|
|              4084|[104]|[108, 109, 110, 8...|
|              4119|[104]|[108, 109, 110, 8...|
|              4227|[104]|[108, 109, 110, 8...|
|              4333|[104]|[108, 109, 110, 8...|
|              4374|[104]|[108, 109, 110, 8...|
|              4651|[104]|[108, 109, 110, 8...|
|              5322|[104]|[108, 109, 110, 8...|
|              5804|[104]|[108, 109, 110, 8...|
|              5901|[104]|[108, 109, 110, 8...|
|              6445|[104]|[108, 109, 110, 8...|
|              6653|[104]|[108, 109, 110, 8...|
|             10158|[104]|[108, 109, 110

In [ ]:
transformed.where(transformed.items == array(*(lit(x) for x in [64, 63, 316]))).show()

+------------------+-------------+----------+
|SalesTransactionID|        items|prediction|
+------------------+-------------+----------+
|             10510|[64, 63, 316]|      [20]|
+------------------+-------------+----------+



## LOGISTIC REGRESSION

In [ ]:
heart_data = pd.read_csv("heart.csv", header=None)

In [ ]:
heart_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [ ]:
schema = ["age", 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

In [ ]:
df = spark.createDataFrame(heart_data, schema)

In [ ]:
df.show()

+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+------+
| age|sex| cp|trestbps| chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+------+
|63.0|1.0|1.0|   145.0|233.0|1.0|    2.0|  150.0|  0.0|    2.3|  3.0|0.0| 6.0|     0|
|67.0|1.0|4.0|   160.0|286.0|0.0|    2.0|  108.0|  1.0|    1.5|  2.0|3.0| 3.0|     2|
|67.0|1.0|4.0|   120.0|229.0|0.0|    2.0|  129.0|  1.0|    2.6|  2.0|2.0| 7.0|     1|
|37.0|1.0|3.0|   130.0|250.0|0.0|    0.0|  187.0|  0.0|    3.5|  3.0|0.0| 3.0|     0|
|41.0|0.0|2.0|   130.0|204.0|0.0|    2.0|  172.0|  0.0|    1.4|  1.0|0.0| 3.0|     0|
|56.0|1.0|2.0|   120.0|236.0|0.0|    0.0|  178.0|  0.0|    0.8|  1.0|0.0| 3.0|     0|
|62.0|0.0|4.0|   140.0|268.0|0.0|    2.0|  160.0|  0.0|    3.6|  3.0|2.0| 3.0|     3|
|57.0|0.0|4.0|   120.0|354.0|0.0|    0.0|  163.0|  1.0|    0.6|  1.0|0.0| 3.0|     0|
|63.0|1.0|4.0|   130.0|254.0|0.0|    2.0|  147.0|  0.0

In [ ]:
df = df.withColumn( 'target', when(df.target > 0,1).otherwise(0))
df.show()

+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+------+
| age|sex| cp|trestbps| chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+------+
|63.0|1.0|1.0|   145.0|233.0|1.0|    2.0|  150.0|  0.0|    2.3|  3.0|0.0| 6.0|     0|
|67.0|1.0|4.0|   160.0|286.0|0.0|    2.0|  108.0|  1.0|    1.5|  2.0|3.0| 3.0|     1|
|67.0|1.0|4.0|   120.0|229.0|0.0|    2.0|  129.0|  1.0|    2.6|  2.0|2.0| 7.0|     1|
|37.0|1.0|3.0|   130.0|250.0|0.0|    0.0|  187.0|  0.0|    3.5|  3.0|0.0| 3.0|     0|
|41.0|0.0|2.0|   130.0|204.0|0.0|    2.0|  172.0|  0.0|    1.4|  1.0|0.0| 3.0|     0|
|56.0|1.0|2.0|   120.0|236.0|0.0|    0.0|  178.0|  0.0|    0.8|  1.0|0.0| 3.0|     0|
|62.0|0.0|4.0|   140.0|268.0|0.0|    2.0|  160.0|  0.0|    3.6|  3.0|2.0| 3.0|     1|
|57.0|0.0|4.0|   120.0|354.0|0.0|    0.0|  163.0|  1.0|    0.6|  1.0|0.0| 3.0|     0|
|63.0|1.0|4.0|   130.0|254.0|0.0|    2.0|  147.0|  0.0

In [ ]:
from pyspark.ml.feature import VectorAssembler
freture_cols = ["age", 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
vec_assembler = VectorAssembler(inputCols = freture_cols, outputCol = "features")




In [ ]:
final_data = vec_assembler.transform(df)
final_data.show()

+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+
| age|sex| cp|trestbps| chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|            features|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+
|63.0|1.0|1.0|   145.0|233.0|1.0|    2.0|  150.0|  0.0|    2.3|  3.0|0.0| 6.0|     0|[63.0,1.0,1.0,145...|
|67.0|1.0|4.0|   160.0|286.0|0.0|    2.0|  108.0|  1.0|    1.5|  2.0|3.0| 3.0|     1|[67.0,1.0,4.0,160...|
|67.0|1.0|4.0|   120.0|229.0|0.0|    2.0|  129.0|  1.0|    2.6|  2.0|2.0| 7.0|     1|[67.0,1.0,4.0,120...|
|37.0|1.0|3.0|   130.0|250.0|0.0|    0.0|  187.0|  0.0|    3.5|  3.0|0.0| 3.0|     0|[37.0,1.0,3.0,130...|
|41.0|0.0|2.0|   130.0|204.0|0.0|    2.0|  172.0|  0.0|    1.4|  1.0|0.0| 3.0|     0|[41.0,0.0,2.0,130...|
|56.0|1.0|2.0|   120.0|236.0|0.0|    0.0|  178.0|  0.0|    0.8|  1.0|0.0| 3.0|     0|[56.0,1.0,2.0,120...|
|62.0|0.0|4.0|   140.0|268.0|0.0|    

In [ ]:
training, testing = final_data.randomSplit([0.7,0.3],seed = 42)

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression( labelCol="target", featuresCol="features")
lrModel = lr.fit(training)

In [ ]:
prediction = lrModel.transform(testing)

In [ ]:
prediction.show()

+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+--------------------+--------------------+----------+
| age|sex| cp|trestbps| chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|            features|       rawPrediction|         probability|prediction|
+----+---+---+--------+-----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+--------------------+--------------------+----------+
|40.0|1.0|4.0|   110.0|167.0|0.0|    2.0|  114.0|  1.0|    2.0|  2.0|0.0| 7.0|     1|[40.0,1.0,4.0,110...|[-3.2748079773596...|[0.03644560814225...|       1.0|
|42.0|1.0|4.0|   140.0|226.0|0.0|    0.0|  178.0|  0.0|    0.0|  1.0|0.0| 3.0|     0|[42.0,1.0,4.0,140...|[1.78117986814546...|[0.85584249458715...|       0.0|
|43.0|1.0|4.0|   150.0|247.0|0.0|    0.0|  171.0|  0.0|    1.5|  1.0|0.0| 3.0|     0|[43.0,1.0,4.0,150...|[1.05329686898155...|[0.74140748579649...|       0.0|
|44.0|1.0|2.0|   120.0|263.0|0.0|    0.0

In [ ]:
final_predictions = prediction.select("prediction", "target")

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
evaluator = BinaryClassificationEvaluator( labelCol= "target", rawPredictionCol="prediction")
evaluator.evaluate(final_predictions)

0.8222641509433962

##Linear Regression

In [ ]:
import pandas as pd
usa_housing = pd.read_csv("USA_Housing.csv")

In [ ]:
usa_housing.describe()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5.000000e+03
mean,68583.108984,5.977222,6.987792,3.981330,36163.516039,1.232073e+06
std,10657.991214,0.991456,1.005833,1.234137,9925.650114,3.531176e+05
min,17796.631190,2.644304,3.236194,2.000000,172.610686,1.593866e+04
25%,61480.562388,5.322283,6.299250,3.140000,29403.928702,9.975771e+05
50%,68804.286404,5.970429,7.002902,4.050000,36199.406689,1.232669e+06
75%,75783.338666,6.650808,7.665871,4.490000,42861.290769,1.471210e+06
max,107701.748378,9.519088,10.759588,6.500000,69621.713378,2.469066e+06


In [ ]:
df = spark.createDataFrame(usa_housing)

In [ ]:
df.show()

+------------------+-------------------+-------------------------+----------------------------+------------------+------------------+--------------------+
|  Avg. Area Income|Avg. Area House Age|Avg. Area Number of Rooms|Avg. Area Number of Bedrooms|   Area Population|             Price|             Address|
+------------------+-------------------+-------------------------+----------------------------+------------------+------------------+--------------------+
| 79545.45857431678|  5.682861321615587|        7.009188142792237|                        4.09|23086.800502686456|1059033.5578701235|208 Michael Ferry...|
| 79248.64245482568| 6.0028998082752425|        6.730821019094919|                        3.09| 40173.07217364482|  1505890.91484695|188 Johnson Views...|
|61287.067178656784|  5.865889840310001|        8.512727430375099|                        5.13| 36882.15939970458|1058987.9878760849|9127 Elizabeth St...|
| 63345.24004622798| 7.1882360945186425|        5.586728664827653|    

In [ ]:
df.printSchema()

root
 |-- Avg. Area Income: double (nullable = true)
 |-- Avg. Area House Age: double (nullable = true)
 |-- Avg. Area Number of Rooms: double (nullable = true)
 |-- Avg. Area Number of Bedrooms: double (nullable = true)
 |-- Area Population: double (nullable = true)
 |-- Price: double (nullable = true)
 |-- Address: string (nullable = true)



In [ ]:
df=df.drop(col("Address"))

In [ ]:
df.show()

+------------------+-------------------+-------------------------+----------------------------+------------------+------------------+
|  Avg. Area Income|Avg. Area House Age|Avg. Area Number of Rooms|Avg. Area Number of Bedrooms|   Area Population|             Price|
+------------------+-------------------+-------------------------+----------------------------+------------------+------------------+
| 79545.45857431678|  5.682861321615587|        7.009188142792237|                        4.09|23086.800502686456|1059033.5578701235|
| 79248.64245482568| 6.0028998082752425|        6.730821019094919|                        3.09| 40173.07217364482|  1505890.91484695|
|61287.067178656784|  5.865889840310001|        8.512727430375099|                        5.13| 36882.15939970458|1058987.9878760849|
| 63345.24004622798| 7.1882360945186425|        5.586728664827653|                        3.26| 34310.24283090706|1260616.8066294468|
| 59982.19722570803|  5.040554523106283|        7.839387785120

In [ ]:
df = df.withColumnRenamed('Avg. Area Income', 'Avg_Area_Income')
df = df.withColumnRenamed('Avg. Area House Age', 'Avg_Area_House_Age')
df = df.withColumnRenamed('Avg. Area Number of Rooms', 'Avg_Area_Number_of_Rooms')
df = df.withColumnRenamed('Avg. Area Number of Bedrooms', 'Avg_Area_Number_of_Bedrooms')
df = df.withColumnRenamed('Area Population', 'Area_Population')

In [ ]:
df.show()
df1=df

+------------------+------------------+------------------------+---------------------------+------------------+------------------+
|   Avg_Area_Income|Avg_Area_House_Age|Avg_Area_Number_of_Rooms|Avg_Area_Number_of_Bedrooms|   Area_Population|             Price|
+------------------+------------------+------------------------+---------------------------+------------------+------------------+
| 79545.45857431678| 5.682861321615587|       7.009188142792237|                       4.09|23086.800502686456|1059033.5578701235|
| 79248.64245482568|6.0028998082752425|       6.730821019094919|                       3.09| 40173.07217364482|  1505890.91484695|
|61287.067178656784| 5.865889840310001|       8.512727430375099|                       5.13| 36882.15939970458|1058987.9878760849|
| 63345.24004622798|7.1882360945186425|       5.586728664827653|                       3.26| 34310.24283090706|1260616.8066294468|
| 59982.19722570803| 5.040554523106283|       7.839387785120487|                   

In [ ]:
from pyspark.ml.feature import VectorAssembler
freture_cols = ['Avg_Area_Income' , 'Avg_Area_House_Age', 'Avg_Area_Number_of_Rooms', 'Avg_Area_Number_of_Bedrooms', 'Area_Population']
vec_assembler = VectorAssembler(inputCols = freture_cols, outputCol = "features")

In [ ]:
final_data = vec_assembler.transform(df)
final_data.show()

+------------------+------------------+------------------------+---------------------------+------------------+------------------+--------------------+
|   Avg_Area_Income|Avg_Area_House_Age|Avg_Area_Number_of_Rooms|Avg_Area_Number_of_Bedrooms|   Area_Population|             Price|            features|
+------------------+------------------+------------------------+---------------------------+------------------+------------------+--------------------+
| 79545.45857431678| 5.682861321615587|       7.009188142792237|                       4.09|23086.800502686456|1059033.5578701235|[79545.4585743167...|
| 79248.64245482568|6.0028998082752425|       6.730821019094919|                       3.09| 40173.07217364482|  1505890.91484695|[79248.6424548256...|
|61287.067178656784| 5.865889840310001|       8.512727430375099|                       5.13| 36882.15939970458|1058987.9878760849|[61287.0671786567...|
| 63345.24004622798|7.1882360945186425|       5.586728664827653|                       3

In [ ]:
training, testing = final_data.randomSplit([0.7,0.3],seed = 42)

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression(featuresCol='features', labelCol='Price', maxIter=10, regParam=0.3, elasticNetParam=0.8)
#lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(training)

In [ ]:
trainingSummary = lrModel.summary

In [ ]:
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 100953.785834
r2: 0.916547


In [ ]:
from pyspark.ml.fpm import FPGrowth

df = spark.createDataFrame([
    (0, [1, 2, 5]),
    (1, [1, 2, 3, 5]),
    (2, [1, 2])
], ["id", "items"])

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(df)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df).show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+---------+----+
|    items|freq|
+---------+----+
|      [1]|   3|
|      [2]|   3|
|   [2, 1]|   3|
|      [5]|   2|
|   [5, 2]|   2|
|[5, 2, 1]|   2|
|   [5, 1]|   2|
+---------+----+

+----------+----------+------------------+----+------------------+
|antecedent|consequent|        confidence|lift|           support|
+----------+----------+------------------+----+------------------+
|    [5, 2]|       [1]|               1.0| 1.0|0.6666666666666666|
|       [2]|       [1]|               1.0| 1.0|               1.0|
|       [2]|       [5]|0.6666666666666666| 1.0|0.6666666666666666|
|    [2, 1]|       [5]|0.6666666666666666| 1.0|0.6666666666666666|
|       [5]|       [2]|               1.0| 1.0|0.6666666666666666|
|       [5]|       [1]|               1.0| 1.0|0.6666666666666666|
|    [5, 1]|       [2]|               1.0| 1.0|0.6666666666666666|
|       [1]|       [2]|               1.0| 1.0|               1.0|
|       [1]|       [5]|0.6666666666666666| 1.0|0.6666666666666666|
+-------

In [ ]:
import pandas as pd
colnames=['items',] 

dfx = pd.read_csv("/content/msnbc990928.csv", names=colnames,header=None)

In [ ]:
dfx.head()

,items
0,1 1
1,2
2,3 2 2 4 2 2 2 3 3
3,5
4,1


In [ ]:
from pyspark.ml.fpm import FPGrowth
dfx=spark.createDataFrame(x)
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(dfx)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(dfx).show()

IllegalArgumentException: ignored

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Loads data.
dataset = spark.createDataFrame(dfx)

# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

IllegalArgumentException: ignored

##Graph Frame

In [ ]:
# creating a simple Graph
id, name
1, Dhaka
2, Khulna
3, Gazipur


In [ ]:
from graphframes import *
from pyspark.sql import SQLContext

sqlContex = SQLContex(sc)
#vetices
v = sqlContex.createDataFrame([(1, "Dhaka"), (2, "Khulna"), (3, "Gazipur")],["id","name"])
#edges
e = sqlContex.createDataFrame([(1,2,350),(1,3,30),(2,1,400),(2,3,450),(3,1,35),(3,2,500)],["src","dist","distance"])

g = GraphFrame(v, e)
g.vertices.show()


NameError: ignored